<a href="https://colab.research.google.com/github/preetmodi/civil-engg-ranking/blob/master/Bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
print("Upload the file college_cutoff.pdf")
uploaded = files.upload()


Saving college_cutoffs.pdf to college_cutoffs.pdf


In [4]:
!pip install tabula-py

     |████████████████████████████████| 11.7MB 2.7MB/s 


In [ ]:
!pip install pypdf2

In [61]:
import PyPDF2
import re
import tabula
import io
import pandas as pd
import numpy as np

# open the pdf file
filename = "college_cutoffs.pdf"
object = PyPDF2.PdfFileReader(io.BytesIO(uploaded[filename]))

# get number of pages
NumPages = object.getNumPages()

# define keyterms
college = " Savitribai Phule Pune University"
print("This may take some while, Please wait!")branch = "Civil Engineering"
count = 0         # counter to count number of entries to average upon
pglist  = []      
college_list = []
rank = []
# extract text and do the search
for i in range(0, NumPages):
  PageObj = object.getPage(i)
  
  Text = PageObj.extractText() 
  # print(Text)
  Search_college = re.search(college, Text) 
  if Search_college is not None:
    Search_branch = re.search(branch, Text)
    if Search_branch is not None:
      # pglist.append(i+1)
      college_name = Text.split("-")[2].split("\n")[0]
      processed_name = re.search("[0-9]", college_name)
      if processed_name is not None:
        college_list.append(college_name[:processed_name.start()])
      else:
        college_list.append(college_name)

      table= tabula.read_pdf(io.BytesIO(uploaded[filename]), pages = i+1, multiple_tables=True)
      count = 0
      ranking=0
      for j in range(len(table)):
        try:
          table[j].drop(table[j].columns[[0]], axis = 1, inplace = True) 
        except:
          pass
      
        for column in table[j].columns:
          if table[j][column][0] is np.nan:
            continue
          else:
            if type(table[j][column][0])==str:
              # print(table[j][column][0].split("\r")[0])
              ranking+=int(table[j][column][0].split("\r")[0])
              count+=1
            else:
              pass
      rank.append(ranking/count)
  

In [62]:
df = pd.DataFrame(rank, columns=['Rank'])
df['College_name'] = college_list
df

,Rank,College_name
0,46945.714286,Nashik District Maratha Vidya Prasarak Samaj'...
1,57638.263158,"Sandip Foundation, Sandip Institute of Techno..."
2,33950.045455,K. K. Wagh Institute of Engineering Education...
3,44826.680000,K. K. Wagh Institute of Engineering Education...
4,62218.615385,Jagadamba Education Soc. Nashik's S.N.D. Coll...
...,...,...
74,62455.666667,"Shri.Someshwar Shikshan Prasarak Mandals, Som..."
75,52593.117647,"Bharati Vidyapeeth's College of Engineering,L..."
76,57080.000000,"Dr. D.Y.Patil Institute of Engineering, Manag..."
77,51449.050000,Pimpri Chinchwad Education Trust's Pimpri Chi...


In [63]:
df = df.sort_values(by=['Rank']).reset_index(drop = True)
df[:10]

,Rank,College_name
0,6128.680000,"College of Engineering, Pune"
1,22538.945205,"MIT Academy of Engineering,Alandi, Pune"
2,22804.466667,"Government College of Engineering & Research,..."
3,23692.012195,B.R.A.C.T's Vishwakarma Institute of Informat...
4,26277.208333,"Pimpri Chinchwad Education Trust, Pimpri Chin..."
5,33950.045455,K. K. Wagh Institute of Engineering Education...
6,37162.683333,"Jaywant Shikshan Prasarak Mandal's,Rajarshi S..."
7,38544.102941,Sanjivani Rural Education Society's Sanjivani...
8,42126.733333,All India Shri Shivaji Memorial Society's Col...
9,42917.714286,G.H.Raisoni College of Engineering & Manageme...
